In [1]:
!pip install datasets
!pip install transformers
!pip install torch

In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
from transformers import AutoTokenizer,AutoModelForMaskedLM
cp = "bert-base-multilingual-uncased"
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import datasets
ds=datasets.load_dataset("csv", data_files='train_df_no_niqqud.csv',sep="\t")
dsv=datasets.load_dataset("csv", data_files='valid_df_no_niqqud.csv',sep="\t")

Using custom data configuration default-66d144cf64d2204f
Found cached dataset csv (/home/eldark/.cache/huggingface/datasets/csv/default-66d144cf64d2204f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-37505e917a9305ac
Found cached dataset csv (/home/eldark/.cache/huggingface/datasets/csv/default-37505e917a9305ac/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def tokenize_function(dataset):
  return tokenizer(str(dataset["verse"])
                      )

tokenized_ds=ds.map(tokenize_function)
tokenized_dsv=dsv.map(tokenize_function)

Loading cached processed dataset at /home/eldark/.cache/huggingface/datasets/csv/default-66d144cf64d2204f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e4ddd62cffd0a70a.arrow
Loading cached processed dataset at /home/eldark/.cache/huggingface/datasets/csv/default-37505e917a9305ac/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0125161d0e5aee94.arrow


In [6]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [7]:
from transformers import TrainingArguments


batch_size = 16
# Show the training loss with every epoch
logging_steps = len(ds["train"])

training_args = TrainingArguments(
    output_dir="Models/Embibert-finetuned-mBert-50-epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-6,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,
    num_train_epochs=50,
)


In [8]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_dsv,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [9]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [10]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

Is the GPU available? True


In [11]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: name, Unnamed: 0.1, verse, verse_idx, Unnamed: 0. If name, Unnamed: 0.1, verse, verse_idx, Unnamed: 0 are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/home/eldark/.local/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 22144
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 69200
  Number of trainable parameters = 167463831
You're using a BertTokenizerFast tokenizer. Please note that with a 

Epoch,Training Loss,Validation Loss
1,3.129500,2.997658
2,2.909300,2.753987
3,2.823500,2.608306
4,2.729900,2.631846
5,2.677700,2.584712
6,2.633600,2.651768
7,2.591200,2.589389
8,2.561200,2.633995
9,2.527300,2.450572
10,2.503300,2.490621


Saving model checkpoint to Models/Embibert-finetuned-mBert-50-epochs/checkpoint-500
Configuration saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-500/config.json
Model weights saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-500/special_tokens_map.json
Saving model checkpoint to Models/Embibert-finetuned-mBert-50-epochs/checkpoint-1000
Configuration saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-1000/config.json
Model weights saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-1000/tokenizer_config.json
Special tokens file saved in Models/Embibert-finetuned-mBert-50-epochs/checkpoint-1000/special_tokens_map.json
The fol

TrainOutput(global_step=69200, training_loss=2.358364024961615, metrics={'train_runtime': 19954.7868, 'train_samples_per_second': 55.485, 'train_steps_per_second': 3.468, 'total_flos': 3.202048584297907e+16, 'train_loss': 2.358364024961615, 'epoch': 50.0})

In [12]:
trainer.save_model("Models/Embibert-finetuned-mBert-50-epochs")
model.save_pretrained("Models/Embibert-finetuned-mBert-50-epochs")

Saving model checkpoint to Models/Embibert-finetuned-mBert-50-epochs
Configuration saved in Models/Embibert-finetuned-mBert-50-epochs/config.json
Model weights saved in Models/Embibert-finetuned-mBert-50-epochs/pytorch_model.bin
tokenizer config file saved in Models/Embibert-finetuned-mBert-50-epochs/tokenizer_config.json
Special tokens file saved in Models/Embibert-finetuned-mBert-50-epochs/special_tokens_map.json
Configuration saved in Models/Embibert-finetuned-mBert-50-epochs/config.json
Model weights saved in Models/Embibert-finetuned-mBert-50-epochs/pytorch_model.bin
